In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import TextNode
from datasets import load_dataset
import uuid
from dotenv import load_dotenv
load_dotenv()
import os
from llama_index.llms.google_genai import GoogleGenAI

embed_model = HuggingFaceEmbedding(model_name="bkai-foundation-models/vietnamese-bi-encoder")

# Indexing


In [ ]:
nodes = []
def _load_dataset(path="khanglt0004/vietnamese_legal_chunks"):
    dataset = load_dataset(path)
    for item in dataset['train']:
        new_node = TextNode(
            text=item['text'],
            id_=str(uuid.uuid5(uuid.NAMESPACE_DNS, str(item['id']))),
            metadata=item['metadata']
        )
        nodes.append(new_node)
    print("Đã tải dữ liệu các chunks, số lượng: ", len(nodes))
_load_dataset()

Đã tải dữ liệu các chunks, số lượng:  3245


In [ ]:

llm = GoogleGenAI(
    model="gemini-2.0-flash",
)

In [ ]:
%pip install llama-index-vector-stores-qdrant

In [ ]:
%pip install -U qdrant_client

In [ ]:



import qdrant_client

client = qdrant_client.QdrantClient(
    "https://958364eb-7484-4a89-afae-40702093201e.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)


vector_store = QdrantVectorStore(
    client=client, 
    aclient=aclient,
    collection_name="law_documents",
    enable_hybrid=True,
    fastembed_sparse_model="Qdrant/bm25",
    batch_size=20,
    )

storage_context = StorageContext.from_defaults(vector_store=vector_store)
# NOTE: reate Vector Database in Qdrant
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model = embed_model,
)

loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=Settings.embed_model
)


In [ ]:
loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=Settings.embed_model
)

In [ ]:
%pip install sentence-transformers
%pip install fastembed
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [18]:
query_engine = index.as_query_engine(
    similarity_top_k=2, sparse_top_k=12, vector_store_query_mode="hybrid", llm=llm
)
from IPython.display import display, Markdown

response = query_engine.query(
    "What is a gangster and his wife? "
)

display(Markdown(str(response)))

They are characters in four tales of violence and redemption.
